### Logs
* 30.08: use 2048 for max_len and max_position_embeddings
* 03.09: use 512 instead

In [2]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q
!pip install accelerate -U -q

In [12]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("deepset/deberta-v3-base-squad2")

from datasets import Dataset, disable_progress_bar
import pandas as pd

pdf = pd.read_csv("./input/prompts_train.csv")
sdf = pd.read_csv("./input/summaries_train.csv")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /deepset/deberta-v3-base-squad2/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1a160a00a0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: c94ca67f-8f8f-42eb-a795-2a6583ae2744)')' thrown while requesting HEAD https://huggingface.co/deepset/deberta-v3-base-squad2/resolve/main/tokenizer_config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /deepset/deberta-v3-base-squad2/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1a160c0df0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 9d04c5b2-ad83-4716-8f80-3e11ab53b7db)')' thrown while requesting HEAD https://huggingface.co/deepset/deberta-v3-base-squad2/resolve/main/config.json


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like deepset/deberta-v3-base-squad2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [4]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large")
## use the pretrained model
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('./input/pretrain/pretrained_model/')
# model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model', config = config)

## Train

In [14]:
%%writefile train.py

import os
import logging
import warnings
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    set_seed,
    Trainer,
    TrainingArguments,
    HfArgumentParser,
    DataCollatorWithPadding,
)
from datasets import Dataset, disable_progress_bar
import pandas as pd
import numpy as np

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_PROJECT'] = 'kaggle-commonlit-eval-student-summaries-2309'

disable_progress_bar()

@dataclass
class Config:
    model_name_or_path: Optional[str] = field(
        default="deepset/deberta-v3-base-squad2",
        metadata={"help": "Model name or path"},
    )

    data_dir: Optional[str] = field(
        default="/kaggle/input/commonlit-evaluate-student-summaries",
        metadata={"help": "Data directory"},
    )

    max_seq_length: Optional[int] = field(
        default=2048,
        #default = 512,
        #default = 512,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=2048,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    )


# Spell auto correction
# from spellchecker import SpellChecker

# def correct_spelling(input_text):
#     print('input_text: ', input_text)
#     # Initialize the spell checker
#     spell = SpellChecker()
#     # Split the input text into words
#     words = input_text.split()
#     # Initialize an empty list to store the corrected words
#     corrected_words = []
#     for word in words:
#         # Check if the word has any punctuation at the end
#         if word[-1].isalpha():
#             # Extract the punctuation
#             punctuation_end = ""
#         else:
#             punctuation_end = word[-1]
#             word = word[:-1]
#         # check if the word has any punctuation at the start
#         if word[0].isalpha():
#            # Extract the punctuation
#             punctuation_start = ""
#         else:
#             punctuation_start = word[0]
#             word = word[1:]
#         # Check the spelling of the word (case insensitive)
#         corrected_word = spell.correction(word.lower())
#         # Preserve the original capitalization
#         if word[0].isupper():
#             corrected_word = corrected_word.capitalize()
#         # Combine the corrected word and punctuation (if any)
#         corrected_word = punctuation_start+corrected_word+punctuation_end
#         # Append the corrected word to the list
#         corrected_words.append(corrected_word)
#     # Join the corrected words back into a single string
#     corrected_text = " ".join(corrected_words)
#     return corrected_text

def tokenize(example, tokenizer, config):
    sep = tokenizer.sep_token

    # if config.add_prompt_question:
    #     text = sep.join(
    #         [example["prompt_question"], example["prompt_text"], example["text"]]
    #     )
    # elif config.add_prompt_text:
    #     text = sep.join([example["prompt_text"], example["text"]])
    # else:
    #     text = example["text"]
    prompt = sep.join([example["prompt_title"], example["prompt_text"], example["prompt_question"]])
    labels = [example["content"], example["wording"]]

    tokenized = tokenizer(
#         prompt,
#         example["text"],
        example['text'],
        prompt,
        padding=False,
        truncation=True, # changed by Peng, turn on the truncation
        max_length=config.max_seq_length,
    )

    return {
        **tokenized,
        "labels": labels,
    }




def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }


def main():
    parser = HfArgumentParser((Config, TrainingArguments))

    config, training_args = parser.parse_args_into_dataclasses()

    set_seed(training_args.seed)

    if "wandb" in training_args.report_to:
        import wandb

        try:
#             from kaggle_secrets import UserSecretsClient
#             user_secrets = UserSecretsClient()
#             key = user_secrets.get_secret("wandb")

#             wandb.login(key=key)
            wandb.login()
        except:
            print("Could not log in to WandB")

    tokenizer = AutoTokenizer.from_pretrained(config.model_name_or_path)
    model_config = AutoConfig.from_pretrained(config.model_name_or_path)

    model_config.update({
        "hidden_dropout_prob": config.dropout,
        "attention_probs_dropout_prob": config.dropout,
        "num_labels": 2,
        "problem_type": "regression",
        "max_position_embeddings": config.max_position_embeddings,
        "cfg": config.__dict__,
    })

    print(model_config)
    
    # Do not use pretrained model
#     model = AutoModelForSequenceClassification.from_pretrained(
#         config.model_name_or_path, config=model_config
#     )
    
    # use pretrained model
    print('use pretrained_model')
    print('deepset/deberta-v3-base-squad2')
    model = AutoModelForSequenceClassification.from_pretrained('deepset/deberta-v3-base-squad2', config = model_config)

    #pdf = pd.read_csv(f"{config.data_dir}/prompts_train.csv")
    pdf = pd.read_csv(f"./input/prompts_train.csv")
    #sdf = pd.read_csv(f"{config.data_dir}/summaries_train.csv")
    sdf = pd.read_csv(f"./input/summaries_train.csv")

    df = pdf.merge(sdf, on="prompt_id")

    # 4 prompt ids, 4 folds
    id2fold = {
        "814d6b": 0,
        "39c16e": 1,
        "3b9047": 2,
        "ebad26": 3,
    }

    df["fold"] = df["prompt_id"].map(id2fold)

    train_ds = Dataset.from_pandas(df[df["fold"] != config.fold])
    val_ds = Dataset.from_pandas(df[df["fold"] == config.fold])

    train_ds = train_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    val_ds = val_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        pad_to_multiple_of=16 if training_args.fp16 else None,
    )

    training_args.bf16 =True
    training_args.gradient_accumulation_steps = 1
    training_args.load_best_model_at_end = True
    training_args.greater_is_better = False
    training_args.metric_for_best_model = 'mcrmse'
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,
    )

    trainer.train()

    model.config.best_metric = trainer.state.best_metric
    model.config.save_pretrained(training_args.output_dir)

    trainer.log({"eval_best_mcrmse": trainer.state.best_metric})


if __name__ == "__main__":
    main()

Overwriting train.py


In [15]:
from pathlib import Path

seed = 42

fold = 1

output = f"output_fold{fold}_seed{seed}_2309"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 2e-6 \
  --weight_decay 0.01 \
  --warmup_ratio 0 \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 1,
    "max_position_embeddings": 2048,
    "max_seq_length": 2048,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 2048,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'loss': 0.4131, 'learning_rate': 1.8578700078308535e-06, 'epoch': 0.29}
{'loss': 0.2974, 'learning_rate': 1.8559122944400938e-06, 'epoch': 0.29}
{'loss': 0.6816, 'learning_rate': 1.8539545810493342e-06, 'epoch': 0.29}
{'eval_loss': 0.47322937846183777, 'eval_content_rmse': 0.7217957377433777, 'eval_wording_rmse': 0.6522803902626038, 'eval_mcrmse': 0.6870380640029907, 'eval_runtime': 113.6607, 'eval_samples_per_second': 18.098, 'eval_steps_per_second': 9.053, 'epoch': 0.29}
{'loss': 0.3255, 'learning_rate': 1.8519968676585747e-06, 'epoch': 0.3}
{'loss': 0.2094, 'learning_rate': 1.850039154267815e-06, 'epoch': 0.3}
{'loss': 0.3232, 'learning_rate': 1.8480814408770556e-06, 'epoch': 0.31}
{'loss': 0.6484, 'learning_rate': 1.846123727486296e-06, 'epoch': 0.31}
{'loss': 0.3174, 'learning_rate': 1.8441660140955364e-06, 'epoch': 0.31}
{'loss': 0.4347, 'learning_rate': 1.8422083007047769e-06, 'epoch': 0.32}
{'loss': 0.33, 'learning_rate': 1.8402505873140171e-06, 'epoch': 0.32}
{'loss': 0.3083,

{'loss': 0.2336, 'learning_rate': 1.677956147220047e-06, 'epoch': 0.65}
{'eval_loss': 0.28906726837158203, 'eval_content_rmse': 0.42977070808410645, 'eval_wording_rmse': 0.6272410750389099, 'eval_mcrmse': 0.5285059213638306, 'eval_runtime': 113.6029, 'eval_samples_per_second': 18.107, 'eval_steps_per_second': 9.058, 'epoch': 0.65}
{'loss': 0.2123, 'learning_rate': 1.6759984338292875e-06, 'epoch': 0.65}
{'loss': 0.4233, 'learning_rate': 1.6740407204385277e-06, 'epoch': 0.65}
{'loss': 0.2712, 'learning_rate': 1.6720830070477681e-06, 'epoch': 0.66}
{'loss': 0.2833, 'learning_rate': 1.6701252936570086e-06, 'epoch': 0.66}
{'loss': 0.3002, 'learning_rate': 1.668167580266249e-06, 'epoch': 0.67}
{'loss': 0.2293, 'learning_rate': 1.6662098668754893e-06, 'epoch': 0.67}
{'loss': 0.2247, 'learning_rate': 1.6642521534847297e-06, 'epoch': 0.67}
{'loss': 0.4393, 'learning_rate': 1.6622944400939702e-06, 'epoch': 0.68}
{'loss': 0.3161, 'learning_rate': 1.6603367267032106e-06, 'epoch': 0.68}
{'loss': 0.

{'loss': 0.3115, 'learning_rate': 1.499804228660924e-06, 'epoch': 1.0}
{'loss': 0.1326, 'learning_rate': 1.4978465152701645e-06, 'epoch': 1.01}
{'loss': 0.1442, 'learning_rate': 1.4958888018794047e-06, 'epoch': 1.01}
{'loss': 0.2478, 'learning_rate': 1.4939310884886452e-06, 'epoch': 1.01}
{'loss': 0.2467, 'learning_rate': 1.4919733750978856e-06, 'epoch': 1.02}
{'loss': 0.201, 'learning_rate': 1.490015661707126e-06, 'epoch': 1.02}
{'loss': 0.2244, 'learning_rate': 1.4880579483163665e-06, 'epoch': 1.03}
{'loss': 0.2346, 'learning_rate': 1.486100234925607e-06, 'epoch': 1.03}
{'loss': 0.1766, 'learning_rate': 1.4841425215348474e-06, 'epoch': 1.03}
{'loss': 0.2421, 'learning_rate': 1.4821848081440874e-06, 'epoch': 1.04}
{'loss': 0.2267, 'learning_rate': 1.4802270947533279e-06, 'epoch': 1.04}
{'loss': 0.357, 'learning_rate': 1.4782693813625683e-06, 'epoch': 1.05}
{'loss': 0.1792, 'learning_rate': 1.4763116679718088e-06, 'epoch': 1.05}
{'loss': 0.2469, 'learning_rate': 1.4743539545810492e-06,

{'loss': 0.512, 'learning_rate': 1.3196945967110415e-06, 'epoch': 1.36}
{'loss': 0.1221, 'learning_rate': 1.3177368833202818e-06, 'epoch': 1.37}
{'loss': 0.1063, 'learning_rate': 1.3157791699295222e-06, 'epoch': 1.37}
{'loss': 0.2414, 'learning_rate': 1.3138214565387626e-06, 'epoch': 1.37}
{'loss': 0.2717, 'learning_rate': 1.311863743148003e-06, 'epoch': 1.38}
{'loss': 0.1708, 'learning_rate': 1.3099060297572435e-06, 'epoch': 1.38}
{'loss': 0.2684, 'learning_rate': 1.307948316366484e-06, 'epoch': 1.39}
{'loss': 0.1763, 'learning_rate': 1.3059906029757244e-06, 'epoch': 1.39}
{'loss': 0.1318, 'learning_rate': 1.3040328895849647e-06, 'epoch': 1.39}
{'loss': 0.1978, 'learning_rate': 1.3020751761942051e-06, 'epoch': 1.4}
{'loss': 0.382, 'learning_rate': 1.3001174628034456e-06, 'epoch': 1.4}
{'loss': 0.1778, 'learning_rate': 1.298159749412686e-06, 'epoch': 1.41}
{'loss': 0.3783, 'learning_rate': 1.2962020360219265e-06, 'epoch': 1.41}
{'eval_loss': 0.2491009384393692, 'eval_content_rmse': 0.4

...

{'loss': 0.1876, 'learning_rate': 7.776037588097102e-07, 'epoch': 2.45}
{'loss': 0.0741, 'learning_rate': 7.756460454189506e-07, 'epoch': 2.45}
{'loss': 0.1934, 'learning_rate': 7.736883320281911e-07, 'epoch': 2.45}
{'loss': 0.3942, 'learning_rate': 7.717306186374315e-07, 'epoch': 2.46}
{'loss': 0.1878, 'learning_rate': 7.697729052466719e-07, 'epoch': 2.46}
{'loss': 0.2278, 'learning_rate': 7.678151918559122e-07, 'epoch': 2.47}
{'eval_loss': 0.2420121729373932, 'eval_content_rmse': 0.41167542338371277, 'eval_wording_rmse': 0.5608457922935486, 'eval_mcrmse': 0.4862605929374695, 'eval_runtime': 113.4523, 'eval_samples_per_second': 18.131, 'eval_steps_per_second': 9.07, 'epoch': 2.47}
{'loss': 0.2246, 'learning_rate': 7.658574784651527e-07, 'epoch': 2.47}
{'loss': 0.1939, 'learning_rate': 7.63899765074393e-07, 'epoch': 2.47}
{'loss': 0.1709, 'learning_rate': 7.619420516836334e-07, 'epoch': 2.48}
{'loss': 0.179, 'learning_rate': 7.599843382928739e-07, 'epoch': 2.48}
{'loss': 0.1416, 'learn

{'loss': 0.258, 'learning_rate': 5.955364134690681e-07, 'epoch': 2.81}
{'loss': 0.137, 'learning_rate': 5.935787000783085e-07, 'epoch': 2.82}
{'loss': 0.1303, 'learning_rate': 5.916209866875489e-07, 'epoch': 2.82}
{'eval_loss': 0.24916236102581024, 'eval_content_rmse': 0.41875743865966797, 'eval_wording_rmse': 0.5683015584945679, 'eval_mcrmse': 0.4935294985771179, 'eval_runtime': 113.4179, 'eval_samples_per_second': 18.136, 'eval_steps_per_second': 9.073, 'epoch': 2.82}
{'loss': 0.1225, 'learning_rate': 5.896632732967893e-07, 'epoch': 2.82}
{'loss': 0.0693, 'learning_rate': 5.877055599060298e-07, 'epoch': 2.83}
{'loss': 0.1149, 'learning_rate': 5.857478465152701e-07, 'epoch': 2.83}
{'loss': 0.1487, 'learning_rate': 5.837901331245106e-07, 'epoch': 2.83}
{'loss': 0.1089, 'learning_rate': 5.818324197337509e-07, 'epoch': 2.84}
{'loss': 0.1417, 'learning_rate': 5.798747063429914e-07, 'epoch': 2.84}
{'loss': 0.1444, 'learning_rate': 5.779169929522317e-07, 'epoch': 2.85}
{'loss': 0.1774, 'lea

{'eval_loss': 0.2409738451242447, 'eval_content_rmse': 0.421987920999527, 'eval_wording_rmse': 0.5512475371360779, 'eval_mcrmse': 0.4866177439689636, 'eval_runtime': 113.5482, 'eval_samples_per_second': 18.116, 'eval_steps_per_second': 9.062, 'epoch': 3.17}
{'loss': 0.1405, 'learning_rate': 4.13469068128426e-07, 'epoch': 3.18}
{'loss': 0.0789, 'learning_rate': 4.115113547376664e-07, 'epoch': 3.18}
{'loss': 0.1407, 'learning_rate': 4.095536413469068e-07, 'epoch': 3.18}
{'loss': 0.17, 'learning_rate': 4.075959279561472e-07, 'epoch': 3.19}
{'loss': 0.124, 'learning_rate': 4.056382145653876e-07, 'epoch': 3.19}
{'loss': 0.0861, 'learning_rate': 4.0368050117462806e-07, 'epoch': 3.19}
{'loss': 0.1039, 'learning_rate': 4.017227877838684e-07, 'epoch': 3.2}
{'loss': 0.1083, 'learning_rate': 3.997650743931088e-07, 'epoch': 3.2}
{'loss': 0.1577, 'learning_rate': 3.9780736100234924e-07, 'epoch': 3.21}
{'loss': 0.2833, 'learning_rate': 3.9584964761158964e-07, 'epoch': 3.21}
{'loss': 0.1528, 'learnin

{'loss': 0.104, 'learning_rate': 2.3727486296006263e-07, 'epoch': 3.53}
{'loss': 0.0913, 'learning_rate': 2.3531714956930305e-07, 'epoch': 3.53}
{'loss': 0.1553, 'learning_rate': 2.3335943617854344e-07, 'epoch': 3.54}
{'loss': 0.1088, 'learning_rate': 2.3140172278778386e-07, 'epoch': 3.54}
{'loss': 0.1225, 'learning_rate': 2.2944400939702428e-07, 'epoch': 3.54}
{'loss': 0.0951, 'learning_rate': 2.2748629600626467e-07, 'epoch': 3.55}
{'loss': 0.1145, 'learning_rate': 2.255285826155051e-07, 'epoch': 3.55}
{'loss': 0.082, 'learning_rate': 2.235708692247455e-07, 'epoch': 3.56}
{'loss': 0.1657, 'learning_rate': 2.216131558339859e-07, 'epoch': 3.56}
{'loss': 0.1152, 'learning_rate': 2.196554424432263e-07, 'epoch': 3.56}
{'loss': 0.1689, 'learning_rate': 2.1769772905246672e-07, 'epoch': 3.57}
{'loss': 0.2409, 'learning_rate': 2.1574001566170711e-07, 'epoch': 3.57}
{'loss': 0.1079, 'learning_rate': 2.137823022709475e-07, 'epoch': 3.57}
{'loss': 0.1217, 'learning_rate': 2.1182458888018793e-07, 

{'loss': 0.0786, 'learning_rate': 5.736100234925607e-08, 'epoch': 3.89}
{'loss': 0.1217, 'learning_rate': 5.540328895849648e-08, 'epoch': 3.89}
{'loss': 0.1653, 'learning_rate': 5.344557556773688e-08, 'epoch': 3.9}
{'loss': 0.1742, 'learning_rate': 5.1487862176977284e-08, 'epoch': 3.9}
{'loss': 0.1202, 'learning_rate': 4.95301487862177e-08, 'epoch': 3.9}
{'loss': 0.092, 'learning_rate': 4.7572435395458104e-08, 'epoch': 3.91}
{'loss': 0.1625, 'learning_rate': 4.5614722004698504e-08, 'epoch': 3.91}
{'loss': 0.1496, 'learning_rate': 4.365700861393892e-08, 'epoch': 3.92}
{'loss': 0.1371, 'learning_rate': 4.1699295223179324e-08, 'epoch': 3.92}
{'loss': 0.1071, 'learning_rate': 3.974158183241973e-08, 'epoch': 3.92}
{'loss': 0.1161, 'learning_rate': 3.7783868441660145e-08, 'epoch': 3.93}
{'loss': 0.1526, 'learning_rate': 3.5826155050900545e-08, 'epoch': 3.93}
{'loss': 0.1774, 'learning_rate': 3.386844166014095e-08, 'epoch': 3.94}
{'eval_loss': 0.24442057311534882, 'eval_content_rmse': 0.41636

In [17]:
#df.loc[0, 'prompt_text']

In [18]:
# from pathlib import Path
# import json

# seed = 42
# scores = []
# for fold in range(4):
#     output = f"./output_fold{fold}_seed{seed}_3008"
#     p = Path(output) / "config.json"

#     with open(p) as fp:
#         cfg = json.load(fp)


#     print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
#     scores.append(cfg['best_metric'])
# print(sum(scores)/len(scores))